This time around you will solve the MDP defined in the last part. You will get that ice cream! But first, we need a bit more learnings:

# Discounted Future Reward

Why do we need the discount factor $\gamma$? The total reward that our agent will receive from the current time step t to the end of the task can be defined as:

$$R_t = r_t + r_{t + 1} + \ldots + r_n$$

That looks ok, but let's not forget that our environment is stochastic (the supermarket might close any time now). The discount factor allows us to value short-term reward more than long-term ones, we can use it as:

$$R_t = R_t + \gamma r_{t+1} + \ldots + \gamma^{n - t} r_n = r_t + \gamma R_{t+1}$$

Our agent would perform great if he chooses the action that maximizes the (discounted) future reward at every step.

# Value function

It would be great to know how "good" a given state $s$ is. Something to tell us - no matter the state you're in if you transition to state $s$ your total reward will be $x$, word! If you start from $s$ and follow policy $\pi$. That would spare us from revisiting same states over and over again. The **value function** does this for us. It depends on the state we're in $s$ and the policy $\pi$ our agent is following. It is given by:

$$V^{\pi}(s) = \mathbb{E}(\sum_{t \geq 0}\gamma^t r_t) \quad \forall s \in \mathbb{S}$$

There exists an **optimal value function** that has the highest value for all states. It is given by:

$$V^*(s) = \max_{\pi}V^{\pi}(s) \quad \forall s \in \mathbb{S}$$

# Q function

Yet, your agent can't control what state he ends up in, directly. He can influence it by choosing some action $a$.  Let's introduce another function that accepts state and action as parameters and returns the expected total reward - the Q function (it represents the "quality" of a certain action given a state). More formally, the function $Q^{\pi}(s, a)$ gives the expected return when starting in $s$, performing $a$ and following $\pi$.

Again, we can define the optimal Q-function $Q^*(s, a)$ that gives the expected total reward for our agent when starting at $s$ and picks action $a$. That is, the optimal Q-function tells our agent how good of a choice is picking $a$ when at state $s$.

There is a relationship between the two optimal functions $V^*$ and $Q^*$. It is given by:

$$V^*(s) = \max_aQ^*(s, a) \quad \forall s \in \mathbb{S}$$

That is, the maximum expected total reward when starting at $s$ is the maximum of $Q^*(s, a)$ over all possible actions.

Using $Q^*(s, a)$ we can extract the optimal policy $\pi^*$ by choosing the action $a$ that gives maximum reward $Q^*(s, a)$ for state $s$. We have:

$$\pi^*(s) = \text{arg}\max_{a} Q^* (s, a) \quad \forall s \in \mathbb{S}$$

There is a nice relationship between all functions we defined so far. You now have the tools to identify states and state-action pairs as good or bad. More importantly, if you can identify $V^*$ or $Q^*$, you can build the best possible agent there is (for the current environment). But how do we use this in practice?

# Learning with Q-learning

Let's focus on a single state $s$ and action $a$. We can express $Q(s, a)$ recursively, in terms of the Q-value of the next state $s'$:

$$Q(s, a) = r + \gamma \max_{a'}Q(s', a')$$

This equation, known as the **Bellman equation**, tells us that the maximum future reward is the reward the agent received for entering the current state $s$ plus the maximum future reward for the next state $s'$. The gist of Q-learning is that we can iteratively approximate $Q^*$ using the Bellman equation described above. The Q-learning equation is given by:

$$Q_{t+1}(s_t, a_t) = Q_t(s_t, a_t) + \alpha(r_{t+1} + \gamma \max_{a}Q_t(s_{t + 1}, a) - Q_t(s_t, a_t))$$

where $\alpha$ is the learning rate that controls how much the difference between previous and new Q-value is considered.

Can our agent learn anything using this? At first - no, the initial approximations will most likely be completely random/wrong. However, as the agent explore more and more of the environment, the approximated Q-values will start to converge to $Q^*$.

# Building the Environment

Okay, it is time to get your ice cream. Let's try a simple case first:

![](./2.rl_for_hackers_part_1_files/small_grid.png)

Let's define our state:

In [1]:
ZOMBIE = "z"
CAR = "c"
ICE_CREAM = "i"
EMPTY = "*"

grid = [
    [ICE_CREAM, EMPTY],
    [ZOMBIE, CAR]
]

In [2]:
for row in grid:
    print(' '.join(row))

i *
z c


In [3]:
from collections import namedtuple

class State:
    
    def __init__(self, grid, car_pos):
        self.grid = grid
        self.car_pos = car_pos
        
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.car_pos == other.car_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.car_pos))
    
    def __str__(self):
        return f"State(grid={self.grid}, car_pos={self.car_pos})"

All possible actions:

In [4]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

ACTIONS = [UP, DOWN, LEFT, RIGHT]

and the initial state:

In [27]:
start_state = State(grid=grid, car_pos=[1, 1])

Your agent needs a way to interact with the environment, that is, choose actions. Let's define a function that takes the current state with an action and returns new state, reward and whether or not the episode has completed:

In [5]:
from copy import deepcopy

def act(state, action):
    
    def new_car_pos(state, action):
        p = deepcopy(state.car_pos)
        if action == UP:
            p[0] = max(0, p[0] - 1)
        elif action == DOWN:
            p[0] = min(len(state.grid) - 1, p[0] + 1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(len(state.grid[0]) - 1, p[1] + 1)
        else:
            raise ValueError(f"Unknown action {action}")
        return p
            
    p = new_car_pos(state, action)
    grid_item = state.grid[p[0]][p[1]]
    
    new_grid = deepcopy(state.grid)
    
    if grid_item == ZOMBIE:
        reward = -100
        is_done = True
        new_grid[p[0]][p[1]] += CAR
    elif grid_item == ICE_CREAM:
        reward = 1000
        is_done = True
        new_grid[p[0]][p[1]] += CAR
    elif grid_item == EMPTY:
        reward = -1
        is_done = False
        old = state.car_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = CAR
    elif grid_item == CAR:
        reward = -1
        is_done = False
    else:
        raise ValueError(f"Unknown grid item {grid_item}")
    
    return State(grid=new_grid, car_pos=p), reward, is_done

In our case, one episode is starting from the initial state and crashing into a Zombie or eating the ice cream.

# Learning to drive

Ok, it is time to implement the Q-learning algorithm and get the ice cream. We have a really small state space, only 4 states. This allows us to keep things simple and store the computed Q values in a table. Let's start with some constants:

In [66]:
import numpy as np
import random

random.seed(42) # for reproducibility

N_STATES = 4
N_EPISODES = 20

MAX_EPISODE_STEPS = 100

MIN_ALPHA = 0.02

alphas = np.linspace(1.0, MIN_ALPHA, N_EPISODES)
gamma = 1.0
eps = 0.2

q_table = dict()

In [67]:
def q(state, action=None):
    
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
        
    if action is None:
        return q_table[state]
    
    return q_table[state][action]

In [68]:
def choose_action(state):
    if random.uniform(0, 1) < eps:
        return random.choice(ACTIONS) 
    else:
        return np.argmax(q(state))

In [69]:
for e in range(N_EPISODES):
    
    state = start_state
    total_reward = 0
    alpha = alphas[e]
    
    for _ in range(MAX_EPISODE_STEPS):
        action = choose_action(state)
        next_state, reward, done = act(state, action)
        total_reward += reward
        
        q(state)[action] = q(state, action) + \
                alpha * (reward + gamma *  np.max(q(next_state)) - q(state, action))
        state = next_state
        if done:
            break
    print('Episode #%d - Total reward = %d.' %(e+1, total_reward))

Episode #1 - Total reward = 999.
Episode #2 - Total reward = 998.
Episode #3 - Total reward = 997.
Episode #4 - Total reward = 997.
Episode #5 - Total reward = 999.
Episode #6 - Total reward = 999.
Episode #7 - Total reward = 998.
Episode #8 - Total reward = -100.
Episode #9 - Total reward = -101.
Episode #10 - Total reward = 999.
Episode #11 - Total reward = 999.
Episode #12 - Total reward = 999.
Episode #13 - Total reward = 999.
Episode #14 - Total reward = 999.
Episode #15 - Total reward = 999.
Episode #16 - Total reward = 998.
Episode #17 - Total reward = 999.
Episode #18 - Total reward = 999.
Episode #19 - Total reward = 999.
Episode #20 - Total reward = 999.


In [70]:
q(start_state)

array([ 998.9999565 ,  225.12936017,  -85.10182825,  586.19245204])

In [71]:
ns, r, done = act(start_state, UP)

In [72]:
r

-1

In [25]:
str(ns)

"State(grid=[['i', 'c'], ['z', '*']], car_pos=[0, 1])"

In [26]:
q(ns)

array([  999.,   998.,  1000.,   999.])

Isn't this amazing? Our agent doesn't know anything about the "rules of the game", yet it manages to learn that Zombies are bad and Ice Cream is great! Also, it tries to reach the ice cream as quickly as possible. The reward is the ultimate signal that drives the learning process.